In [150]:
!pip install torch
import torch

import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

zsh:1: command not found: pip


In [151]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [152]:
class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"


  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out

  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward

    return out

  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')

    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward

    return out


  def backward(self):

    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [153]:
class Neuron:
  def __init__(self, nin):
    self.w =[Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  def __call__(self, x):
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  def parameters(self):
    return self.w + [self.b]

class Layer:
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs
  def parameters(self):
    return [p for neurons in self.neurons for p in neurons.parameters()]

class MLP:
  def __init__(self, nin, nouts):
    sz = [nin] + nouts    # add the input dimension as first layer and after that each layer will have previous layer n neurons output as n dimentional input , the last layer will have only one neuron so that we can have one prediction it can also have mutiple outputs but here it iwll be one.
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    for layer in self.layers:
      x= layer(x)
    return x;
  def parameters(self):
    return [p for layers in self.layers for p in layers.parameters()]

In [154]:
xs = [
    # [1.11, 1.12, -1.13]
    [2.0,3.0,-1.0],
    [3.0,-1.0,0.5],
    [0.5,1.0, 1.0],
    [1.0,1.0,-1.0]
]
n= MLP(3,[4,4,1])
ys = [1.0, -1.0, -1.0, 1.0] # desired outout

yprid = [n(x) for x in xs]
yprid


[[Value(data=-0.98431892135704)],
 [Value(data=-0.5732852958296546)],
 [Value(data=-0.5121756537721991)],
 [Value(data=-0.9774799820136655)]]

In [155]:
loss = sum(((yout[0]-ygt)*(yout[0]-ygt)) for ygt, yout in zip(ys, yprid))
loss

Value(data=8.2680066924481)

In [156]:
loss.backward()


In [157]:
n.layers[0].neurons[0].w[0].grad

0.3733616509205048

In [158]:
n.layers[0].neurons[0].w[0].data

-0.8217314466125483

this slight chnage in weight improvs the the loss dramatically:
forward pass:

In [159]:
yprid = [n(x) for x in xs]
loss = sum(((yout[0]-ygt)*(yout[0]-ygt)) for ygt, yout in zip(ys, yprid))
print(loss.data)
print(yprid)
print(ys)

8.2680066924481
[[Value(data=-0.98431892135704)], [Value(data=-0.5732852958296546)], [Value(data=-0.5121756537721991)], [Value(data=-0.9774799820136655)]]
[1.0, -1.0, -1.0, 1.0]


In [160]:
# now run the backword pass to update the gradients it self.
loss.backward()
#!/bin/bash


update:

now changing grad little bit to minimise the loss
the logic for negative here is
if let's say grad is +ve so increasing the weight will increase the loss so we need to decreae the weights so need -ve sign

if let's say grad is -ve so we wanna increase the weight so that loss can decrease hence we need to make -ve grad sign to +ve hence again -ve *sign*

In [161]:
for p in n.parameters():
  p.data +=-0.01 * p.grad 


In [162]:
for p in n.parameters():
  print(p)
len(n.parameters())
# draw_dot(n(x)[0])

Value(data=-0.8291986796309584)
Value(data=-0.3806013410950249)
Value(data=0.33062924235889446)
Value(data=-0.027131659271635973)
Value(data=-0.9759320384222411)
Value(data=0.4688687525563479)
Value(data=-0.898296093468187)
Value(data=-0.6057774796670697)
Value(data=0.12104942655216483)
Value(data=0.21139648345406584)
Value(data=-0.41286242960004793)
Value(data=-0.286076676911969)
Value(data=0.2573747642752236)
Value(data=-0.6129534820127742)
Value(data=0.6216204827277477)
Value(data=0.47886871654244)
Value(data=0.7599730581778152)
Value(data=-0.5884523889784035)
Value(data=-0.8921611064447186)
Value(data=-0.03373260348612301)
Value(data=-0.33922731320253996)
Value(data=-0.9878497903318578)
Value(data=0.6502602856839987)
Value(data=-0.6348041812018224)
Value(data=-0.6078100133249322)
Value(data=0.9418641872717076)
Value(data=0.5662951140439263)
Value(data=0.3324218240643566)
Value(data=0.7306365882930318)
Value(data=-0.9820590947204811)
Value(data=0.8786451458717488)
Value(data=0.78911

41

In [208]:
for k in range(10):  # max 100 iterations

  # forward pass
  yprid = [n(x) for x in xs]
  loss = sum(((yout[0]-ygt)*(yout[0]-ygt)) for ygt, yout in zip(ys, yprid))
  
  for p in n.parameters():
    p.grad = 0.0

  # Backward pass
  loss.backward()
  
  print(f"Step {k}, Loss: {loss.data}")

  for p in n.parameters():
    p.data += -0.01 * p.grad
  
  
  if loss.data < 0.01:
    print(f"Converged at step {k}!")
    break

print(loss)
print(yprid)
print(ys)

Step 0, Loss: 0.010245129847984046
Step 1, Loss: 0.010214000490325134
Step 2, Loss: 0.010183052368281945
Step 3, Loss: 0.010152283931589895
Step 4, Loss: 0.01012169364742532
Step 5, Loss: 0.010091280000163055
Step 6, Loss: 0.01006104149113747
Step 7, Loss: 0.010030976638407702
Step 8, Loss: 0.010001083976526864
Step 9, Loss: 0.009971362056314758
Converged at step 9!
Value(data=0.009971362056314758)
[[Value(data=0.9603439718143755)], [Value(data=-0.9583808657096059)], [Value(data=-0.9440614231714686)], [Value(data=0.9405232417815804)]]
[1.0, -1.0, -1.0, 1.0]
